In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [21]:
training = spark.read.csv('data_LR.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    sunny| 29|         4| 20000|
|     paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [22]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [23]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [24]:
# group independent features

# ['Age','Experience']

from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['Age','Experience'],outputCol="Independent features")


In [25]:
output = featureassembler.transform(training)

In [26]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    sunny| 29|         4| 20000|          [29.0,4.0]|
|     paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [27]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent features']

In [28]:
finalized_data = output.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [49]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = "Independent Features", labelCol = "Salary")
regressor = regressor.fit(train_data)

In [50]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [21.0,1.0]| 15000|
+--------------------+------+



In [51]:
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [52]:
regressor.intercept

22295.299605312008

In [53]:
pred_results = regressor.evaluate(test_data)

In [54]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17203.085755292603|
+--------------------+------+------------------+



In [55]:
pred_results.meanSquaredError,pred_results.meanAbsoluteError

(4853586.845173177, 2203.0857552926027)